In [ ]:
import os
import requests
import pandas as pd
from datetime import datetime, timedelta
import json
from pathlib import Path
import time

In [ ]:
BASE_URL = "https://api.the-odds-api.com/v4/historical"
API_KEY = os.environ.get("ODDS_API_KEY")
SPORT = "cricket_ipl"

In [ ]:
def get_historical_odds(event_id: str, date: str):
    try:
        url = f"{BASE_URL}/sports/{SPORT}/events/{event_id}/odds"
        params = {
            "apiKey": API_KEY,
            "regions": "us",
            "markets": "h2h",
            "event_id": event_id,
            "bookmakers": "draftkings",
            "date": date
        }
        response = requests.get(url, params=params)
        response.raise_for_status()
        return response.json()
    
    except requests.exceptions.RequestException as e:
        print(e)
        return None

In [ ]:
def get_timestamps(start_str: str, end_str: str, interval_minutes: int = 5) -> list:
    timestamps = []
    if not start_str or not end_str:
        return timestamps
    
    start_time = datetime.strptime(start_str, '%Y-%m-%dT%H:%M:%SZ')
    end_time = datetime.strptime(end_str, '%Y-%m-%dT%H:%M:%SZ')
    
    current = start_time
    while current <= end_time:
        timestamps.append(current.strftime('%Y-%m-%dT%H:%M:%SZ'))
        current += timedelta(minutes=interval_minutes)
    return timestamps

In [ ]:
def get_odds(row: pd.Series) -> None:
    the_odds_api_match_id = row['the_odds_api_id']
    
    output_file = Path("/Users/darshan/Documents/GitHub/ipl-sentiment-trader/the_odds_api/2024") / f"{row['match_number']}.json"
    if output_file.exists():
        print(f"Skipping - data exists")
        return
    
    if row['abandoned'] == 1:
        print(f"Skipping - match abandoned")
        return
        
    timestamps = get_timestamps(row['start'], row['end'])
    if not timestamps:
        print(f"Skipping - no valid timestamps")
        return
        
    odds_data = []
    for ts in timestamps:
        try:
            data = get_historical_odds(the_odds_api_match_id, ts)
            if data:
                odds_data.append({
                    'timestamp': ts,
                    'odds': data
                })
        except Exception as e:
            print(f"Error fetching odds for {the_odds_api_match_id} at {ts}: {str(e)}")
            continue
    
    if odds_data:
        output_file.parent.mkdir(parents=True, exist_ok=True)
        output_file.write_text(json.dumps(odds_data, indent=2))
        print(f"Saved odds for {row['home_team']} vs {row['away_team']}")
    else:
        print(f"No odds data collected for {row['home_team']} vs {row['away_team']}")

In [ ]:
timestamps_df = pd.read_csv('/Users/darshan/Documents/GitHub/ipl-sentiment-trader/the_odds_api/extractor/timestamps.csv')

for _, row in timestamps_df.iterrows():
    print(f"Processing {row['match_number']}: {row['home_team']} vs {row['away_team']}")
    get_odds(row)
    time.sleep(1)